In [3]:
import pandas as pd
import os
import numpy as np
import gc
number_of_statements_int = 13
number_of_statements = str(number_of_statements_int)


In [4]:
import enum
# Enum for size units
class SIZE_UNIT(enum.Enum):
    BYTES = 1
    KB = 2
    MB = 3
    GB = 4
def convert_unit(size_in_bytes, unit):
    """ Convert the size from bytes to other units like KB, MB or GB"""
    if unit == SIZE_UNIT.KB:
        return size_in_bytes/1024
    elif unit == SIZE_UNIT.MB:
        return size_in_bytes/(1024*1024)
    elif unit == SIZE_UNIT.GB:
        return size_in_bytes/(1024*1024*1024)
    else:
        return size_in_bytes
    
def get_file_size(file_name, size_type = SIZE_UNIT.BYTES ):
    """ Get file in size in given unit like KB, MB or GB"""
    size = os.path.getsize(file_name)
    return convert_unit(size, size_type)

In [5]:
count = 0
dictionary_list = []
label_lookup = pd.read_csv("D:\\AMEX\\Test_Train\\train_labels.csv")
for chunk in pd.read_csv("D:\\AMEX\\Test_Train\\train_data.csv", chunksize=19994):
    current_data = chunk
    count = count + 1
    current_data = current_data.merge(label_lookup, on="customer_ID", how="left")
    #####################
    grouped = current_data.groupby(['customer_ID']).size().reset_index(name="count")
    grouped = grouped[grouped['count'] == number_of_statements_int]   
    current_data = current_data.merge(grouped, on="customer_ID", how="inner")
    current_data.to_csv('balanced_train_'+number_of_statements+'_statement.csv', mode='a', header=not os.path.exists('balanced_train_'+number_of_statements+'_statement.csv'), index=False)
    size = get_file_size('balanced_train_'+number_of_statements+'_statement.csv', SIZE_UNIT.GB)
    if count % 10 == 0:
        print("Current Size", size)
    if size > 2.5:
        print("Final Size", size)
        break

    

Current Size 0.4980922434478998
Current Size 0.9969209777191281
Current Size 1.4945815699175
Current Size 1.9948378475382924
Current Size 2.4944254653528333
Final Size 2.5448707565665245


In [6]:
del current_data
gc.collect()

32

In [7]:
balanced_dataset = pd.read_csv("/home/grantredfield/Desktop/AMEX_DEFAULT/balanced_train_"+number_of_statements+"_statement.csv", low_memory=False)


FileNotFoundError: [Errno 2] No such file or directory: '/home/grantredfield/Desktop/AMEX_DEFAULT/balanced_train_13_statement.csv'

In [ ]:
balanced_dataset['target'].value_counts()

In [ ]:
balanced_dataset['S_2'] = pd.to_datetime(balanced_dataset['S_2'])
balanced_dataset['RANK'] = balanced_dataset.groupby('customer_ID')['S_2'].rank(ascending=False)


In [ ]:
balanced_dataset = balanced_dataset.sort_values(['customer_ID', 'S_2'], ascending=[True, True])
balanced_dataset.to_csv('/home/grantredfield/Desktop/AMEX_DEFAULT/balanced_train_'+number_of_statements+'_statement.csv' , index=False)